# Introduction

The following code can be used to 

* Apply mutations and sequences of mutations to quivers
* Given two quivers, search for mutation sequences that can be applied to the first to give the second (allowing for possible relabelling of these quivers) 
* Search for mutation sequences that fix a quiver (up to permutation), proving periodicity
* Given a mutation sequence that fixes a quiver (not up to permutation), generate T- and Y-systems
* Given a mutate sequence that fixes a quiver (up to permutation), generate reduced T- and Y-systems

Any comments or found errors can be sent to me at joepallister@gmail.com

# Contents:
* [Functions](#functions)
* [Mutation and mutation sequences](#mutation)
* [Mutation sequences between pairs of quivers](#findmutations)
* [T-systems](#tsystems)
* [Y-systems](#ysystems)
* [T-systems examples](#tsystemsexample)
* [Y-systems examples](#ysystemsexample)

The functions section can be ignored by people not interested in the implementation here. 

"Mutation and mutation sequences" is two examples of how we apply single and sequences of mutations.

In "Mutation sequences between pairs of quivers" we show how we can search for mutation sequences that get us from one quiver to another (up to permutation). We also note that if we apply this with the same quiver as both arguments then we can find mutation sequences that fix a quiver, so we are able to identify periodic quivers.

In "T-systems" we show how we can generate T-systems from periodic quivers. We give a full presentation of these systems, but there is a convenient way to reduce these systems which we mainly focus on. 

"Y-systems" is analogous to "T-systems" 

In [1]:
import itertools
import numpy as np
from tqdm import tqdm
import timeit
from IPython.display import display, Latex, Math

# Functions <a class="anchor" id="functions"></a>

## Cluster Algebra Functions

In [2]:
def max_plus(num):
    if num <= 0:
        return 0
    return num

def mutate(quiver, vertex):
    dim = quiver.shape[0]
    result = np.zeros(shape=np.shape(quiver))
    for i in range(quiver.shape[0]):
        for j in range(quiver.shape[1]):
            if (i == vertex or j == vertex):
                result[i,j] = -quiver[i,j]
            else:
                result[i,j] = quiver[i,j]+np.sign(quiver[i,vertex])*max_plus(quiver[i,vertex]*quiver[vertex,j])
    return np.array(result)

def mutation_sequence(quiver, sequence):
    current_quiver = quiver
    for vertex in sequence:
        current_quiver = mutate(current_quiver, vertex)
    return current_quiver

def mutation_sequences_between_quivers(quiver1, quiver2, depth, early_stop=True):
    '''
    Return a list of all possible mutation sequences, up to length "depth", that can be apply to quiver1 to give a 
    relabelling of quiver2. If early_stop is true this will stop as soon as it finds a sequence, otherwise it
    will search for all possibilities
    '''
    results = []
    quiver2_permutations = get_permutations(quiver2)
    pairs = {"()":quiver1}    
    vertices = range(quiver1.shape[1])
    
    for i in tqdm(range(1, depth+1)):
        combs = combinations_without_adjacent_duplicates(vertices, i)
        for comb in combs:
            previous_quiver = pairs[repr(comb[:-1])]
            new_quiver = mutate(previous_quiver, comb[-1])
            pairs[repr(comb)] = new_quiver
            if any((new_quiver == quiver).all() for quiver in quiver2_permutations):
                results.append(comb)
                if early_stop:
                    return results
    return results

# Systems

In [3]:
# For latex printing we sometimes need to use triple curly brackets, for example

# f"x_{{{variable_subscripts[row_idx]}}}"

# Here the inner brackets evaluate variable_subscripts[row_idx]. We want to have actual curly brackets appearing in the string
# so our Latex will look like, for example, x_{12}, so we need to stop these curly brackets from evaluating, which is done with
# a pair of brackets {{}}, hence we end up with triple {{{}}}

# Note that for the fraction in the Y-system we need to escape the \ in the \frac in the f-string, this is done with \\

def T_system(quiver, sequence):
    if not sequence:
        display(Latex("No system here due to trivial sequence"))
        return 0
    current_quiver = quiver
    dim = quiver.shape[0]
    labelling_message = f"We write the $j$th cluster variable that appears at vertex $i$ by $x_i(j)$"
    display(Latex(labelling_message))
    frozen_vertices = list(set(range(dim))-set(sequence))
    frozen_message = f"The frozen vertices here are at vertices ${frozen_vertices}$ so we write the variables there as $c_i$, for $i \in {frozen_vertices}$"
    if frozen_vertices:
        display(Latex(frozen_message))
    variable_subscripts = [0] * dim # The initial variable subscripts at each vertex
    for idx, vertex in enumerate(sequence):
        row = current_quiver[:,vertex]
        positive_prod = ""
        negative_prod = ""
        for row_idx, row_val in enumerate(row):
            # If row_val is +ve it contributes to positive_prod. If row_val = 1 we don't need to write the exponent in x^1
            if row_val == 1:
                if row_idx not in frozen_vertices:
                    positive_prod += f"x_{{{row_idx}}}({variable_subscripts[row_idx]})"
                else:
                    positive_prod += f"c_{{{row_idx}}}"
            elif row_val>0:
                if row_idx not in frozen_vertices:   
                    positive_prod += f"x_{{{row_idx}}}({variable_subscripts[row_idx]})^{{{int(row_val)}}}"
                else:
                    positive_prod += f"c_{{{row_idx}}}^{{{int(row_val)}}}"
            # If row_val is ive it contributes to negative_prod. If row_val = -1 we don't need to write the exponent in x^1
            elif row_val == -1:
                if row_idx not in frozen_vertices:
                    negative_prod += f"x_{{{row_idx}}}({variable_subscripts[row_idx]})"
                else:
                    negative_prod += f"c_{{{row_idx}}}"
            elif row_val<0:
                if row_idx not in frozen_vertices:
                    negative_prod += f"x_{{{row_idx}}}({variable_subscripts[row_idx]})^{{{-int(row_val)}}}"
                else:
                    negative_prod += f"c_{{{row_idx}}}^{{{-int(row_val)}}}"
        # If the products are empty we need 1, instead of an empty string
        if positive_prod == "":
            positive_prod = "1"
        if negative_prod == "":
            negative_prod = "1"
        equation = f'$x_{{{vertex}}}({variable_subscripts[vertex]+1})x_{{{vertex}}}({variable_subscripts[vertex]})={positive_prod}+{negative_prod}$'
        display(Math(equation))
        current_quiver = mutate(current_quiver, vertex)
        variable_subscripts[vertex] += 1
        
    if np.array_equal(quiver[0:quiver.shape[1]], current_quiver[0:quiver.shape[1]]):
        if len(set([var for var in variable_subscripts if var != 0])) == 1:
            periodic_message = f"This mutation sequence fixes the quiver, so the above equations hold for $n\in Z$ \
                if we replace all $x_i(n)$ with $x_i(n+{[var for var in variable_subscripts if var != 0][0]})$"
            display(Latex(periodic_message))
        else:
            periodic_message = f"This mutation sequence fixes the quiver but since we mutate at the non-frozen \
                vertices a different number of times the system is given by shifting each of the variables \
                $x_i(n)$ to $x_i(n+v_i)$ where the $v_i$ are given by ${variable_subscripts}$. \
                Any zeroes here correspond to the shifts for frozen vertices"
            display(Latex(periodic_message))
    else:
        non_periodic_message = f"This mutation sequence doesn't fix the quiver, so we don't have a T-system here,\
            just a set of {len(sequence)} equations"
        display(Latex(non_periodic_message)) 
        
def T_system_reduced(quiver, sequence, perm):
    full_sequence = permuted_mutation_sequence(perm, sequence)
    if not full_sequence:
        display(Latex("No system here due to trivial sequence"))
        return 0
    full_sequence = full_sequence + full_sequence
    full_sequence_pairs = enumerate(full_sequence)
    # If we don't do this the the enumerate object can only be used once
    full_sequence_pairs = [pair for pair in full_sequence_pairs]
    current_quiver = quiver
    dim = quiver.shape[0]
    labelling_message = f"We have ${len(sequence)}$ sequences which we write as $w_i(n)$"
    display(Latex(labelling_message))
    n_deep_frozen_vertices = dim - quiver.shape[1]
    frozen_vertices = list(set(range(dim))-set(full_sequence))
    frozen_message = f"The frozen vertices here are at vertices ${frozen_vertices}$ so we write the variables there as $c_i$, for $i \in {frozen_vertices}$"
    if frozen_vertices:
        display(Latex(frozen_message))
    for idx, vertex in enumerate(sequence):
        row = current_quiver[:,vertex]
        positive_prod = ""
        negative_prod = ""
        for row_idx, row_val in enumerate(row):
            if row_val == 0:
                continue
            # If row_val is +ve it contributes to positive_prod. If row_val = 1 we don't need to write the exponent in x^1
            next_mutate = 0
            future_mutations = [pair for pair in full_sequence_pairs if pair[1] == row_idx and pair[0]>idx]
            if future_mutations:
                next_mutate = future_mutations[0][0]
            which_sequence = next_mutate % len(sequence)
            which_idx = next_mutate // len(sequence)
                
            if row_val == 1:
                if future_mutations:
                    positive_prod += f"w_{{{which_sequence}}}({which_idx})"
                else:
                    positive_prod += f"c_{{{row_idx}}}"
            elif row_val>0:
                if future_mutations:   
                    positive_prod += f"w_{{{which_sequence}}}({which_idx})^{{{int(row_val)}}}"
                else:
                    positive_prod += f"c_{{{row_idx}}}^{{{int(row_val)}}}"
            # If row_val is ive it contributes to negative_prod. If row_val = -1 we don't need to write the exponent in x^1
            elif row_val == -1:
                if row_idx not in frozen_vertices:
                    negative_prod += f"w_{{{which_sequence}}}({which_idx})"
                else:
                    negative_prod += f"c_{{{row_idx}}}"
            elif row_val<0:
                if row_idx not in frozen_vertices:
                    negative_prod += f"w_{{{which_sequence}}}({which_idx})^{{{-int(row_val)}}}"
                else:
                    negative_prod += f"c_{{{row_idx}}}^{{{-int(row_val)}}}"
                    
        # If the products are empty we need 1, instead of an empty string
        if positive_prod == "":
            positive_prod = "1"
        if negative_prod == "":
            negative_prod = "1"
            
        this_sequence = idx % len(sequence)
        this_idx = idx // len(sequence)
        next_mutation = [pair for pair in full_sequence_pairs if pair[1] == vertex and pair[0]>idx][0]        
        next_sequence = next_mutation[0] % len(sequence)
        next_idx = next_mutation[0] // len(sequence)
        equation = f'$w_{{{next_sequence}}}({next_idx})w_{{{this_sequence}}}({this_idx})={positive_prod}+{negative_prod}$'
        display(Math(equation))
        current_quiver = mutate(current_quiver, vertex)
    
    final_quiver = 0
    if n_deep_frozen_vertices:
        final_quiver = perm.dot(current_quiver).dot(np.linalg.inv(perm[:-n_deep_frozen_vertices, :-n_deep_frozen_vertices]))
    else:
        final_quiver = perm.dot(current_quiver).dot(np.linalg.inv(perm))
    if np.array_equal(final_quiver, quiver):
        periodic_message = f"This mutation sequence fixes the quiver, so the above equations hold for $n\in Z$ \
            if we replace all $w_i(n)$ with $w_i(n+1)$"
        display(Latex(periodic_message))
    else:
        non_periodic_message = f"This mutation sequence doesn't fix the quiver, so we don't have a T-system here,\
            just a set of {len(sequence)} equations"
        display(Latex(non_periodic_message))

In [39]:
def Y_system(quiver, sequence):
    # It's possible to obtain a trivial sequence (e.g. doing mutations [1,1]) so we exclude this
    if not sequence:
        display(Latex("No system here due to trivial sequence"))
        return 0
    
    num_rows = quiver.shape[0]
    labelling_message = f"We write the coefficient variable that appears at vertex $i$ at mutation $j$ by $y_i(j)$"
    display(Latex(labelling_message))
    
    # Collect all the quivers we obtain from doing the mutation sequence. Actually we need the whole list twice
    # so we can see far enough ahead to write down the system
    quivers = [quiver]
    for vertex in sequence+sequence:
        quivers.append(mutate(quivers[-1], vertex))

    # enumerate(sequence) is Nakanishi's forward mutation points P_+
    for idx, vertex in enumerate(sequence):
        # This searches for the next time we mutate at the current vertex
        next_mutate = ""
        for next_idx, next_vertex in enumerate(sequence+sequence):
            if next_idx <= idx:
                continue
            if next_vertex == vertex:
                next_mutate = next_idx
                break
             
        num_prod = ""
        denom_prod = ""    
        for quiver_idx, inner_vertex in enumerate(sequence+sequence):
            if quiver_idx < idx:
                continue
            if quiver_idx >= next_mutate:
                break
            current_quiver = quivers[quiver_idx]
            num_arrows = int(current_quiver[inner_vertex, vertex])
            if num_arrows == 0:
                continue
                
            elif num_arrows == -1:
                num_prod += f"(1+y_{{{inner_vertex}}}({quiver_idx}))"
            elif num_arrows < -1:
                num_prod += f"(1+y_{{{inner_vertex}}}({quiver_idx}))^{{{-num_arrows}}}"
            elif num_arrows == 1:
                denom_prod += f"(1+y_{{{inner_vertex}}}({quiver_idx})^{{-1}})"
            else:
                denom_prod += f"(1+y_{{{inner_vertex}}}({quiver_idx})^{{-1}})^{{{num_arrows}}}"

        if num_prod == "":
            num_prod = "1"
        equation = f'$y_{{{vertex}}}({next_mutate})y_{{{vertex}}}({idx})=\\dfrac{{{num_prod}}}{{{denom_prod}}}$'
        if denom_prod == "":
            equation = f'$y_{{{vertex}}}({next_mutate})y_{{{vertex}}}({idx})={num_prod}$'
        display(Math(equation))

        
    if np.array_equal(quivers[0], quivers[-1]):
        periodic_message = f"This mutation sequence fixes the quiver, so the above equations hold for $n\in Z$"
        display(Latex(periodic_message))
    else:
        non_periodic_message = f"This mutation sequence doesn't fix the quiver, so we don't have a Y-system here,\
            just a set of {len(sequence)} equations"
        display(Latex(non_periodic_message)) 
        
def Y_system_reduced(quiver, sequence, perm):
    # It's possible to obtain a trivial sequence (e.g. doing mutations [1,1]) so we exclude this
    if not sequence:
        display(Latex("No system here due to trivial sequence"))
        return 0
    
    full_sequence = permuted_mutation_sequence(perm, sequence)
    num_rows = quiver.shape[0]
    labelling_message = f"We write the sequences of coefficient variables as $v_i(j)$"
    display(Latex(labelling_message))
    
    # Collect all the quivers we obtain from doing the mutation sequence. Actually we need the whole list twice
    # so we can see far enough ahead to write down the system
    quivers = [quiver]
    for vertex in full_sequence+full_sequence:
        quivers.append(mutate(quivers[-1], vertex))

    # enumerate(sequence) is Nakanishi's forward mutation points P_+
    for idx, vertex in enumerate(sequence):
        # This searches for the next time we mutate at the current vertex
        next_mutate = ""
        for next_idx, next_vertex in enumerate(full_sequence+full_sequence):
            if next_idx <= idx:
                continue
            if next_vertex == vertex:
                next_mutate = next_idx
                break
        num_prod = ""
        denom_prod = ""    
        for quiver_idx, inner_vertex in enumerate(full_sequence+full_sequence):
            if quiver_idx < idx:
                continue
            if quiver_idx >= next_mutate:
                break
            which_sequence = quiver_idx % len(sequence)
            which_idx = quiver_idx // len(sequence)
            current_quiver = quivers[quiver_idx]
            num_arrows = int(current_quiver[inner_vertex, vertex])
            
            if num_arrows == 0:
                continue
            elif num_arrows == -1:
                num_prod += f"(1+v_{{{which_sequence}}}({which_idx}))"
            elif num_arrows < -1:
                num_prod += f"(1+v_{{{which_sequence}}}({which_idx}))^{{{-num_arrows}}}"
            elif num_arrows == 1:
                denom_prod += f"(1+v_{{{which_sequence}}}({which_idx})^{{-1}})"
            else:
                denom_prod += f"(1+v_{{{which_sequence}}}({which_idx})^{{-1}})^{{{num_arrows}}}"

        this_sequence = idx % len(sequence)
        this_idx = idx // len(sequence)
        next_sequence = next_mutate % len(sequence)
        next_idx = next_mutate // len(sequence)
        if num_prod == "":
            num_prod = "1"
        equation = f'$v_{{{next_sequence}}}({next_idx})v_{{{this_sequence}}}({this_idx})=\\dfrac{{{num_prod}}}{{{denom_prod}}}$'
        if denom_prod == "":
            equation = f'$v_{{{next_sequence}}}({next_idx})v_{{{this_sequence}}}({this_idx})={num_prod}$'
        display(Math(equation))

        
    if np.array_equal(quivers[0], quivers[-1]):
        periodic_message = f"This mutation sequence fixes the quiver, so the above equations hold for $n\in Z$"
        display(Latex(periodic_message))
    else:
        non_periodic_message = f"This mutation sequence doesn't fix the quiver, so we don't have a Y-system here,\
            just a set of {len(sequence)} equations"
        display(Latex(non_periodic_message)) 

# Combinatorics

In [5]:
def perm_action(perm, idx):
    return perm[idx,].nonzero()[0][0]
        
def permuted_mutation_sequence(perm, vertices):
    result = vertices
    current_pair = vertices
    for i in range(order_of_matrix(perm)-1):
        current_pair = [perm_action(perm, vertex) for vertex in current_pair]
        result = result + current_pair
    return result

def which_permutation(quiver1, quiver2):
    num_frozen = quiver1.shape[0] - quiver1.shape[1]
    permutation_mats = [np.array(mat) for mat in list(itertools.permutations(np.eye(quiver1.shape[0])))]
    if num_frozen:
        permutation_mats = [mat for mat in permutation_mats if np.array_equal(mat[-num_frozen:, -num_frozen:], np.eye(num_frozen))]
        pairs = [[mat, mat.dot(quiver1).dot(np.linalg.inv(mat[:-num_frozen, :-num_frozen]))] for mat in permutation_mats]
        return [pair[0] for pair in pairs if np.array_equal(pair[1], quiver2)]
    pairs = [[mat, mat.dot(quiver1).dot(np.linalg.inv(mat))] for mat in permutation_mats]
    return [pair[0] for pair in pairs if np.array_equal(pair[1], quiver2)]

def get_permutations(quiver):
    num_frozen = quiver.shape[0] - quiver.shape[1]
    permutation_mats = [np.array(mat) for mat in list(itertools.permutations(np.eye(quiver.shape[0])))]
    if num_frozen:
        permutation_mats = [mat for mat in permutation_mats if np.array_equal(mat[-num_frozen:, -num_frozen:], np.eye(num_frozen))]
        return [mat.dot(quiver).dot(np.linalg.inv(mat[:-num_frozen, :-num_frozen])) for mat in permutation_mats]
    return [mat.dot(quiver).dot(np.linalg.inv(mat)) for mat in permutation_mats]

def no_adjacent_duplicates(sequence):
    'Returns true if sequence has no adjacent duplicate entries. False otherwise'
    for idx in range(len(sequence)-1):
        if sequence[idx] == sequence[idx+1]:
            return False
    return True

def remove_adjacent_duplicates(sequence):
    result = []
    remainder = sequence
    while remainder:
        if len(remainder) == 1:
            result.append(remainder[0])
            break
        elif remainder[0] == remainder[1]:
            remainder = remainder[2:]
        else:
            result.append(remainder[0])
            remainder = remainder[1:]
    # not clear why this is a special case
    if len(result) == 2 and result[0] == result[1]:
        return []
    return result

def combinations_without_adjacent_duplicates(vertices, depth):
    'Returns all possible sequences without adjacent duplicate vertices of given length (here called depth)'
    all_sequences = itertools.product(vertices, repeat=depth)
    return [sequence for sequence in all_sequences if no_adjacent_duplicates(sequence)]

def order_of_matrix(mat):
    order = 1
    power = mat
    while not np.array_equal(power, np.identity(mat.shape[0])):
        order += 1
        power = np.matmul(power, mat)
    return order

# Mutation and mutation sequences <a class="anchor" id="mutation"></a>

We can apply single mutations and mutation sequences

In [6]:
quiver = np.array([
    [0, 0, -1, 1, 1],
    [0, 0, 1, -1, -1],
    [1, -1, 0, 0, -1],
    [-1, 1, 0, 0, 1],
    [-1, 1, 1, -1, 0]
])

mutate(quiver, 0)

array([[ 0.,  0.,  1., -1., -1.],
       [ 0.,  0.,  1., -1., -1.],
       [-1., -1.,  0.,  1.,  0.],
       [ 1.,  1., -1.,  0.,  1.],
       [ 1.,  1.,  0., -1.,  0.]])

In [7]:
mutation_sequence(quiver, [0, 3, 2])

array([[ 0.,  0., -0.,  1., -1.],
       [ 0.,  0., -0.,  1., -1.],
       [-0., -0., -0.,  1., -1.],
       [-1., -1., -1.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.]])

# Mutation sequences between pairs of quiver <a class="anchor" id="findmutations"></a>

We can use the mutation_sequences_between_quivers functions to search for mutation sequences that, applied to the first quiver, give the second (up to a permutation). This also has a depth argument to determine how many mutations we are allowed to use. It also has an early_stop argument (True by default) that will cause the function to stop when it finds the first mutation sequence.

In this example, the we get the sequences $[0, 3, 2]$ and $[2, 1, 0]$, which, applied to "quiver" should give "quiver2"

In [8]:
quiver2 = np.array([
    [0, 0, 0, 1, -1],
    [0, 0, 0, 1, -1],
    [0, 0, 0, 1, -1],
    [-1, -1, -1, 0, 0],
    [1, 1, 1, 0, 0]
])

mutation_sequences_between_quivers(quiver, quiver2, depth=3, early_stop=False)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 126.23it/s]


[(0, 3, 2), (2, 1, 0)]

We apply these two sequences $[0, 3, 2]$ and $[2, 1, 0]$ to "quiver" to see that, in the first case we do get "quiver2" exactly. In the second case we only get a permuation of "quiver2". We can use the which_permuation function to see which permutations we can use.

In [9]:
mutation_sequence(quiver, [0, 3, 2])

array([[ 0.,  0., -0.,  1., -1.],
       [ 0.,  0., -0.,  1., -1.],
       [-0., -0., -0.,  1., -1.],
       [-1., -1., -1.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.]])

In [10]:
mutation_sequence(quiver, [2, 1, 0])

array([[-0., -1., -0., -0.,  1.],
       [ 1., -0.,  1.,  1.,  0.],
       [-0., -1.,  0.,  0.,  1.],
       [-0., -1.,  0.,  0.,  1.],
       [-1.,  0., -1., -1.,  0.]])

We use the which_permutation function to determine the possibly permutations we can apply to the mutated "quiver" to get "quiver2"

In [11]:
mutated_quiver = mutation_sequence(quiver, [2, 1, 0])
sigmas = which_permutation(mutated_quiver, quiver2)
sigmas

[array([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.]]),
 array([[1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.]]),
 array([[0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.]]),
 array([[0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.]]),
 array([[0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.]]),
 array([[0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.]])]

To check this, we select a permutation and apply it to the mutated "quiver" (done by conjugating). This is the same as "quiver2"

In [12]:
sigma = sigmas[0]
sigma.dot(mutated_quiver).dot(np.linalg.inv(sigma))

array([[ 0.,  0.,  0.,  1., -1.],
       [ 0.,  0.,  0.,  1., -1.],
       [ 0.,  0.,  0.,  1., -1.],
       [-1., -1., -1.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.]])

# Periodic quivers

We can also use our mutation_sequences_between_quivers function to search for sequences that fix a quiver (up to permutation), i.e. we look for periodic quivers. This is done simply by using the same argument for both quivers in the function. If necessary we can find a suitable permutation as above.

Here we give an example with depth=3, so we look for sequences of length up to 3 that fix the quiver.

In [13]:
quiver3 = np.array([
    [0, 1, 0, 0],
    [-1, 0, 1, 0],
    [0, -1, 0, 1],
    [0, 0, -1, 0]
])

mutation_sequences_between_quivers(quiver3, quiver3, depth=3, early_stop=False)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 800.44it/s]


[(0, 2, 3),
 (1, 0, 3),
 (1, 2, 3),
 (1, 3, 0),
 (2, 0, 3),
 (2, 1, 0),
 (2, 3, 0),
 (3, 1, 0)]

# T-systems <a class="anchor" id="tsystems"></a>

We can (attempt) to get a T-system as follows. If the given sequence doesn't fix the quiver then we don't get a recurrent set of equations, just a finite set. If the sequence does fix the quiver then we do get a recurrent system.

In [14]:
T_system(quiver, [0, 3, 2])

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

In [16]:
T_system(quiver3, [0, 2, 3, 2, 1, 0, 1, 3, 2, 3, 0, 1])

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

We saw above that the sequence $[0, 2, 3]$ fixes "quiver3" up to mutation. To generate a full T-system we need the full sequence that fixes the quiver. This can be generated from $[0, 2, 3]$ and our choice of permutation by the get_permuted_mutation_sequence function.

In [17]:
sigma = which_permutation(mutation_sequence(quiver3, [0, 2, 3]), quiver3)[0]
sequence = permuted_mutation_sequence(sigma, [0, 2, 3])
sequence

[0, 2, 3, 2, 1, 0, 1, 3, 2, 3, 0, 1]

This full sequence can be used to generate a full T-system, since it completely fixes the quiver (not up to permutation)

In [18]:
T_system(quiver3, sequence)

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

We can write down a reduced version of this T-system (though other reductions are available)

In [19]:
T_system_reduced(quiver3, [0, 2, 3], sigma)

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

# Y systems <a class="anchor" id="ysystems"></a>

We can obtain full and reduced Y-systems in the same way as we obtained T-systems

In [21]:
quiver3 = np.array([
    [0, 1, 0, 0],
    [-1, 0, 1, 0],
    [0, -1, 0, 1],
    [0, 0, -1, 0]
])

sigma = which_permutation(mutation_sequence(quiver3, [0, 2, 3]), quiver3)[0]
sequence = permuted_mutation_sequence(sigma, [0, 2, 3])
Y_system(quiver3, sequence)

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

In [22]:
Y_system_reduced(quiver3, [0, 2, 3], sigma)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

# T-systems examples <a class="anchor" id="tsystemsexample"></a>

## Quiver $(45)$, left, of https://arxiv.org/pdf/2109.11107.pdf

This quiver has a parameter $p$, which we need to take an arbitrary value of.

We first use mutation_sequences_between_quivers to find sequences which fix this quiver. We then use which_permutation to find an appropriate permutation. Finally we can plug these in to T_system_reduced to get the system, which is equation (63) in the above paper.

In [23]:
p=2

quiver_45 = np.array([
    [0, -1, 0, p, -1],
    [1, 0 ,-1, 0, 0],
    [0, 1, 0, 1, -p],
    [-p, 0, -1, 0, p+1],
    [1, 0, p, -p-1, 0]
])

mutation_sequences_between_quivers(quiver_45, quiver_45, depth=3, early_stop=False)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 125.68it/s]


[(0, 1), (2, 1), (1, 0, 1), (1, 2, 1)]

In [24]:
sigma = which_permutation(mutation_sequence(quiver_45, [0, 1]), quiver_45)[0]
T_system_reduced(quiver_45, [0, 1], sigma)

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

We can also try one of the longer mutation sequences to get a different system.

In [25]:
sigma = which_permutation(mutation_sequence(quiver_45, [1, 2, 1]), quiver_45)[0]
T_system_reduced(quiver_45, [1, 2, 1], sigma)

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

## Okubo's quiver (4.19) of https://arxiv.org/pdf/1505.03067.pdf

In Okubo's work the T-system is given by applying the mutation sequence $[0, 1]$ to this quiver, but the reduction to the shorter T-system is done slightly different to what we do here. In our work Okubo's (4.19) is obtained by applying $[2,3]$. We get a slightly different system by applying $[0, 1]$.

In [26]:
okubo = np.array([
    [0, 0, -1, 1, 1, -1],
    [0, 0, 1, -1, -1, 1],
    [1, -1, 0, 0, -1, 1],
    [-1, 1, 0, 0, 1, -1],
    [-1, 1, 1, -1, 0, 0],
    [1, -1, -1, 1, 0, 0]
])

mutation_sequences_between_quivers(okubo, okubo, depth=4, early_stop=False)

100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.98it/s]


[(0, 1),
 (1, 0),
 (2, 3),
 (3, 2),
 (4, 5),
 (5, 4),
 (0, 1, 0, 1),
 (0, 1, 2, 3),
 (0, 1, 3, 2),
 (0, 1, 4, 5),
 (0, 1, 5, 4),
 (0, 2, 4, 0),
 (0, 2, 4, 1),
 (0, 4, 2, 0),
 (0, 4, 2, 1),
 (1, 0, 1, 0),
 (1, 0, 2, 3),
 (1, 0, 3, 2),
 (1, 0, 4, 5),
 (1, 0, 5, 4),
 (1, 3, 5, 0),
 (1, 3, 5, 1),
 (1, 5, 3, 0),
 (1, 5, 3, 1),
 (2, 0, 4, 2),
 (2, 0, 4, 3),
 (2, 3, 0, 1),
 (2, 3, 1, 0),
 (2, 3, 2, 3),
 (2, 3, 4, 5),
 (2, 3, 5, 4),
 (2, 4, 0, 2),
 (2, 4, 0, 3),
 (3, 1, 5, 2),
 (3, 1, 5, 3),
 (3, 2, 0, 1),
 (3, 2, 1, 0),
 (3, 2, 3, 2),
 (3, 2, 4, 5),
 (3, 2, 5, 4),
 (3, 5, 1, 2),
 (3, 5, 1, 3),
 (4, 0, 2, 4),
 (4, 0, 2, 5),
 (4, 2, 0, 4),
 (4, 2, 0, 5),
 (4, 5, 0, 1),
 (4, 5, 1, 0),
 (4, 5, 2, 3),
 (4, 5, 3, 2),
 (4, 5, 4, 5),
 (5, 1, 3, 4),
 (5, 1, 3, 5),
 (5, 3, 1, 4),
 (5, 3, 1, 5),
 (5, 4, 0, 1),
 (5, 4, 1, 0),
 (5, 4, 2, 3),
 (5, 4, 3, 2),
 (5, 4, 5, 4)]

Okubo's sequence $[0, 1]$ here gives a different presentation of the system to his (4.19)

In [27]:
sigma = which_permutation(mutation_sequence(okubo, [0, 1]), okubo)[3]
T_system_reduced(okubo, [0, 1], sigma)

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

Instead we obtain his (4.19) by doing $[2,3]$

In [28]:
sigma = which_permutation(mutation_sequence(okubo, [2, 3]), okubo)[3]
T_system_reduced(okubo, [2, 3], sigma)

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

We can look at some of the systems from the mutation sequences of length 4

In [29]:
for sigma in which_permutation(mutation_sequence(okubo, [0, 2, 4, 0]), okubo):
    print(sigma)
    T_system_reduced(okubo, [0, 2, 4, 0], sigma)

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

## Quiver 2.21 https://arxiv.org/pdf/2403.00721.pdf

This is an example of a quiver with frozen vertices, which is represented by a rectangular matrix. We look at mutation sequences of depth up to 4.

The above paper's (2.21) doesn't seem to come directly here, but is probably just a different representation of one (or both) of the systems here.

In [31]:
quiver221 = np.array([
    [0, 1, -1, 0, -1, 1],
    [-1, 0, 2, -1, 1, -1],
    [1, -2, 0, 1, 1, -1],
    [0, 1, -1, 0, -1, 1],
    [1, -1, -1, 1, 0, 0],
    [-1, 1, 1, -1, 0, 0],
    [1, 0, 0, -1, 1, -1],
    [-1, -1, 1, 1, 0, 0]
])

mutation_sequences_between_quivers(quiver221, quiver221, depth=4, early_stop=False)

100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.89it/s]


[(0, 1),
 (0, 3),
 (0, 4),
 (3, 0),
 (3, 2),
 (3, 5),
 (4, 5),
 (5, 4),
 (1, 0, 1),
 (2, 3, 2),
 (4, 0, 4),
 (4, 0, 5),
 (5, 3, 4),
 (5, 3, 5),
 (0, 1, 2, 3),
 (0, 1, 3, 2),
 (0, 1, 4, 1),
 (0, 1, 4, 2),
 (0, 1, 4, 5),
 (0, 2, 5, 0),
 (0, 2, 5, 1),
 (0, 2, 5, 3),
 (0, 2, 5, 4),
 (0, 3, 0, 2),
 (0, 3, 0, 3),
 (0, 3, 0, 5),
 (0, 3, 1, 2),
 (0, 3, 2, 1),
 (0, 4, 0, 5),
 (0, 4, 1, 2),
 (0, 4, 1, 4),
 (0, 4, 1, 5),
 (0, 4, 5, 0),
 (0, 5, 2, 0),
 (0, 5, 2, 1),
 (0, 5, 2, 3),
 (0, 5, 2, 4),
 (1, 3, 4, 1),
 (1, 4, 3, 1),
 (2, 0, 5, 2),
 (2, 5, 0, 2),
 (3, 0, 1, 2),
 (3, 0, 2, 1),
 (3, 0, 3, 0),
 (3, 0, 3, 1),
 (3, 0, 3, 4),
 (3, 1, 4, 0),
 (3, 1, 4, 2),
 (3, 1, 4, 3),
 (3, 1, 4, 5),
 (3, 2, 0, 1),
 (3, 2, 1, 0),
 (3, 2, 5, 1),
 (3, 2, 5, 2),
 (3, 2, 5, 4),
 (3, 4, 1, 0),
 (3, 4, 1, 2),
 (3, 4, 1, 3),
 (3, 4, 1, 5),
 (3, 5, 2, 1),
 (3, 5, 2, 4),
 (3, 5, 2, 5),
 (3, 5, 3, 4),
 (3, 5, 4, 3),
 (4, 1, 3, 4),
 (4, 1, 3, 5),
 (4, 3, 1, 4),
 (4, 3, 1, 5),
 (4, 5, 0, 1),
 (4, 5, 0, 3),
 (4, 5, 0, 5),
 

In [32]:
for sigma in which_permutation(mutation_sequence(quiver221, [0, 1]), quiver221):
    print(sigma)
    T_system_reduced(quiver221, [0, 1], sigma)

[[0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

In [33]:
for sigma in which_permutation(mutation_sequence(quiver221, [0, 4]), quiver221):
    print(sigma)
    T_system_reduced(quiver221, [0, 4], sigma)

[[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

We can look at some of the length 3 sequences

In [35]:
for sigma in which_permutation(mutation_sequence(quiver221, [4, 0, 4]), quiver221):
    print(sigma)
    T_system_reduced(quiver221, [4,0,4], sigma)

[[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

Finally we can look at some of the length 4 sequences

# Y-systems examples <a class="anchor" id="ysystemsexample"></a>

## Okubo's quiver (4.19) of https://arxiv.org/pdf/1505.03067.pdf

We looked at the T-system for this quiver above. We can arrive at the reduced Y-system as follows. The long Y-system here should be analoguous to Okubo's (4.23). Note that to agree with Okubo's (4.24) we need to take 

$$v_i(j)\rightarrow \begin{cases}
z_j & i+j \:\:\mathrm{even} \\
y_j & i+j \:\: \mathrm{odd}
\end{cases}$$

in our reduced system.

In [44]:
sigma = which_permutation(mutation_sequence(okubo, [0, 1]), okubo)[3]
sequence = permuted_mutation_sequence(sigma, [0, 1])
Y_system(okubo, sequence)

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

In [40]:
sigma = which_permutation(mutation_sequence(okubo, [0, 1]), okubo)[3]
Y_system_reduced(okubo, [0, 1], sigma)

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Latex object>